## Library and Data

In [ ]:
# import libraries
import os
import pandas as pd
from pandas import DataFrame
import numpy as np
from sklearn.preprocessing import MinMaxScaler
#from sklearn.externals import joblib
import seaborn as sns
sns.set(color_codes=True)
import matplotlib.pyplot as plt
%matplotlib inline

from numpy.random import seed

#tf.logging.set_verbosity(tf.logging.ERROR)
from keras.layers import Input, Dropout, Dense, LSTM, TimeDistributed, RepeatVector
from keras.models import Model
from keras import regularizers
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, precision_recall_curve, cohen_kappa_score, fbeta_score
from sklearn.metrics import recall_score, classification_report, auc, roc_curve, log_loss
LABELS = ["Normal","FMA"]

#set up graphic style in this case I am using the color scheme from xkcd.com
from pylab import rcParams
rcParams['figure.figsize'] = 14, 8.7 # Golden Mean
col_list = ["cerulean","scarlet"]# https://xkcd.com/color/rgb/
sns.set(style='white', font_scale=1.75, palette=sns.xkcd_palette(col_list))

In [ ]:
import time
import joblib
# set random seed
import tensorflow as tf
tf.random.set_seed(123)
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

## Tunes

In [ ]:
from kerastuner import HyperModel
from kerastuner import Hyperband
from kerastuner import BayesianOptimization
from kerastuner.tuners import RandomSearch

In [ ]:
from keras.layers import Input, Dropout, Dense, LSTM, TimeDistributed, RepeatVector
from tensorflow.keras import models, layers
from keras.models import Sequential
from sklearn.preprocessing import StandardScaler

## GPU

In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

## Data

In [ ]:
# Cargar training y testing
# Read in data and display first 5 rows
test = pd.read_csv("Dataset_Test.csv")
# validation
train = pd.read_csv("Dataset_Training.csv")
print('The shape of our train is:', train.shape)
#test.head(5)
#train.head(5)

In [ ]:
train.describe()

In [ ]:
train = train[train['FMA'] == 0] # Seleccion de datos
#train.head()

In [ ]:
split = 1460 # Validation
validation = train[-split:]
train = train[:-split]
# The last element contains the labels
# Convertir Series to DataFrame (.to_frame())
train_fma = train.iloc[:, -1]
validation_fma = validation.iloc[:, -1]
test_fma = test.iloc[:, -1]

# Columnas
train_fma.columns = ['train_fma']
validation_fma.columns = ['validation_fma']
test_fma.columns = ['test_fma']

In [ ]:
# Eliminar Campos
#test = test.drop(['temp_avg','temp_min', 'atmos_pres_min', 'wd', 'atmos_pres_max','atmos_pres_avg', 'rh','ceil_hgt', 'visibility', 'FMA'], axis=1) 
# ceil_hgt # visibility ,'temp_max','temp_min'
test = test.drop(['atmos_pres_min', 'atmos_pres_max','ceil_hgt', 'visibility', 'FMA'], axis=1)

# Eliminar Campos
#train = train.drop(['temp_avg','temp_min', 'atmos_pres_min', 'wd', 'atmos_pres_max','atmos_pres_avg', 'rh','ceil_hgt', 'visibility', 'FMA'], axis=1)
train = train.drop(['atmos_pres_min', 'atmos_pres_max','ceil_hgt', 'visibility', 'FMA'], axis=1)

# Eliminar Campos
#validation = validation.drop(['temp_avg','temp_min', 'atmos_pres_min', 'wd', 'atmos_pres_max','atmos_pres_avg', 'rh','ceil_hgt', 'visibility', 'FMA'], axis=1)
validation = validation.drop(['atmos_pres_min', 'atmos_pres_max','ceil_hgt', 'visibility', 'FMA'], axis=1)

In [ ]:
print("Training dataset shape:", train.shape)
print("Test dataset shape:", validation.shape)
print("Test dataset shape:", test.shape)

## Normalize the data or Standardize the data ?

In [ ]:
# normalize the data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(train)
X_test = scaler.transform(test)
X_validation = scaler.transform(validation)
scaler_filename = "scaler_data"
joblib.dump(scaler, scaler_filename)

In [ ]:
# preprocessing - normalization
scaler = StandardScaler()
scaler.fit(train)
x_train_scaled = scaler.transform(train)
x_test_scaled = scaler.transform(test)
x_validation_scaled = scaler.transform(validation)

In [ ]:
# reshape inputs for LSTM [samples, timesteps, features]
x_train_scaled = x_train_scaled.reshape(x_train_scaled.shape[0], 1, X_train.shape[1]) # X_train 
print("Training data shape:", x_train_scaled.shape)
x_test_scaled = x_test_scaled.reshape(x_test_scaled.shape[0], 1, x_test_scaled.shape[1])     # X_test 
print("Test data shape:", x_test_scaled.shape)
x_validation_scaled = x_validation_scaled.reshape(x_validation_scaled.shape[0], 1, x_validation_scaled.shape[1])  #X_validation
print("Test data shape:", x_validation_scaled.shape)

In [ ]:
class RegressionHyperModel(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape
    def build(self, hp):
        model = Sequential()
        model.add(
            layers.LSTM(8,
                #units=hp.Int('units', 8, 64, 4, default=8),
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'),
                return_sequences=True,
                input_shape=input_shape
            )
        )
        
        L1=model.add(
            layers.LSTM(
                units=hp.Int('units_2', min_value=16, max_value=512, step=16), # 'units_2', 16, 32, 64, default=32),
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'),
                    kernel_regularizer=regularizers.l2(hp.Choice('regularizers.l2', values=[0.00, 0.01, 0.05, 0.1])), # Valorarlo... 
                return_sequences=True,
                input_shape=input_shape
            )
        ) 
        
        L2=model.add(
            layers.LSTM(
                units=hp.Int('units_3', min_value=2, max_value=8, step=1),              #'units_3', 2, 4, 8, default=4
                activation=hp.Choice( 
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'),
                return_sequences=True,
                input_shape=input_shape
            )
        )  
        
        
        L3=model.add(
            layers.LSTM(
                units=hp.Int('units_4', min_value=2, max_value=8, step=1),              #'units_3', 2, 4, 8, default=4
                activation=hp.Choice( 
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'),
                return_sequences=True,
                input_shape=input_shape
            )
        ) 
        
        L4=model.add(
            layers.LSTM(
                units=hp.Int('units_5', min_value=32, max_value=512, step=16),        #'units_4', 16, 32, 64, default=32
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'),
                return_sequences=True,
                input_shape=input_shape
            )
        )
        model.add(
            layers.Dropout(
                hp.Float(
                    'dropout',
                    min_value=0.0,
                    max_value=0.1,
                    default=0.005,
                    step=0.01)
            )
        )
        
        model.add(layers.Dense(8))
        optimizer = hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop','nadam', 'adadelta',
                                            'adagrad', 'adamax','ftrl'])
        model.compile(
            optimizer=optimizer,loss='mse',metrics=['mse']
        )
        
        return model

## Instanciar HyperModel

In [ ]:
input_shape = (x_train_scaled.shape[1],x_train_scaled.shape[2],)
hypermodel = RegressionHyperModel(input_shape)

## Búsqueda aleatoria

In [ ]:
tuner_rs = RandomSearch(
            hypermodel,
            objective='mse',
            seed=42,
            max_trials= 10, # 20
            project_name='helloworld',
            overwrite=True,
            executions_per_trial=4)

In [ ]:
tuner_rs.search(x_train_scaled, x_train_scaled, epochs=20, validation_data=(x_validation_scaled, x_validation_scaled), verbose=0)

In [ ]:
tuner_rs.search_space_summary()

In [ ]:
tuner_rs.results_summary()

In [ ]:
best_model = tuner_rs.get_best_models(num_models=1)[0]
loss, mse = best_model.evaluate(x_test_scaled, x_test_scaled)

In [ ]:
best_model = tuner_rs.get_best_models(num_models=1)[0]
loss, mse = best_model.evaluate(x_train_scaled, x_train_scaled)

In [ ]:
tuner_rs.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values

In [ ]:
best_model = tuner_rs.get_best_models(num_models=1)[0]
mse_rs = best_model.evaluate(x_test_scaled, x_test_scaled)[1]
print('Random search MSE: ', mse_rs)

## Hiperbanda

In [ ]:
tuner_hb = Hyperband(
            hypermodel,
            max_epochs= 20,
            objective='mse',
            seed=42,
            overwrite=True,
            executions_per_trial=1
        )

In [ ]:
tuner_hb.search(x_train_scaled, x_train_scaled, epochs= 100, validation_data=(x_validation_scaled, x_validation_scaled), verbose=0) # y_train
best_model = tuner_hb.get_best_models(num_models=1)[0]
best_model.evaluate(x_test_scaled, x_test_scaled) # y_test

In [ ]:
tuner_hb.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values

In [ ]:
best_model = tuner_hb.get_best_models(num_models=1)[0]
mse_hb = best_model.evaluate(x_test_scaled, x_test_scaled)[1]

In [ ]:
print('Hyperband Optimization MSE: ', mse_hb)

## Bayesian Optimization

In [ ]:
tuner_bo = BayesianOptimization(
            hypermodel,
            objective='mse',
            max_trials=2,
            seed=42,
            overwrite=True,
            executions_per_trial=2
        )

In [ ]:
tuner_bo.search(x_train_scaled, x_train_scaled, epochs=100, validation_data=(x_validation_scaled, x_validation_scaled), verbose=0) 

In [ ]:
best_model = tuner_bo.get_best_models(num_models=1)[0]

In [ ]:
best_model.evaluate(x_test_scaled, x_test_scaled)

In [ ]:
tuner_bo.get_best_models(num_models=1)[0]

In [ ]:
tuner_bo.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values

In [ ]:
best_model = tuner_bo.get_best_models(num_models=1)[0]
mse_bo = best_model.evaluate(x_test_scaled, x_test_scaled)[1]

In [ ]:
print('Bayesian Optimization MSE: ', mse_bo)

In [ ]:
print('Hyperband Optimization MSE: ', mse_hb)

In [ ]:
print('Random search MSE: ', mse_rs)

In [ ]:
tuner_bo.get_best_models()[0].summary()

## Modelos

In [ ]:
tuner_rs.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values

In [ ]:
tuner_rs.get_best_models()[0].summary()

In [ ]:
tuner_bo.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values

In [ ]:
tuner_bo.get_best_models()[0].summary()

In [ ]:
tuner_hb.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values

In [ ]:
tuner_hb.get_best_models()[0].summary()